<a href="https://colab.research.google.com/github/thoadao0301/AI-Project/blob/NamTP/Implement_Keras_Facenet_Vietnam_Famous_Celebrities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount data and extract dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare dataset, remove images that has more than two face

In [ ]:
!pip install keras_facenet

In [ ]:
import sys
import os, random
import numpy as np
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from keras_facenet import FaceNet

In [ ]:
def check_only_one_face(directory_path):
    list_sub_dir = os.listdir(directory_path)
    # # create the detector, using default weights
    embedder = FaceNet()
    for sub_dir in list_sub_dir:
        dir_path = directory_path
        image_path = os.path.join(dir_path, sub_dir)
        print(image_path)
        list_file = os.listdir(image_path)
        with tqdm(total=len(list_file), file=sys.stdout) as pbar:

            for file_name in list_file:
                file_path = os.path.join(image_path, file_name)
                # load image from file
                try:
                  image = Image.open(file_path)
                except:
                  try:
                    os.remove(file_path)
                    continue
                  except:
                    print('Can\'t not delete file:', file_path)
                    continue

                # convert to RGB, if needed
                image = image.convert('RGB')
                # convert to array
                pixels = np.asarray(image)
                # detect faces in the image
                results = embedder.extract(pixels)
                if len(results) != 1:
                    try:
                      os.remove(file_path)
                    except:
                      print('Can\'t not delete file:', file_path)
                pbar.update(1)

In [ ]:
directory_path = ''
check_only_one_face(directory_path)

In [ ]:
directory_path = ''
check_only_one_face(directory_path)

# Create test dataset

In [ ]:
from shutil import move

def move_data_to_path(src, des, num=30):
    list_sub_dir = os.listdir(src)
    for sub_dir in list_sub_dir:
        print(sub_dir)
        dir_path = src
        dir_copy_path = des
        image_path = os.path.join(dir_path, sub_dir)
        image_copy_path = os.path.join(dir_copy_path, sub_dir)
        try:
            os.mkdir(image_copy_path)
        except:
            continue
        list_file = os.listdir(image_path)
        i = 1
        with tqdm(total=num, file=sys.stdout) as pbar:
            for file_name in list_file:
                file_path = os.path.join(image_path, file_name)
                file_copy_path = os.path.join(image_copy_path,file_name)
                move(file_path,file_copy_path)
                pbar.update(1)
                i+=1
                if i > num:
                  break
        
    

In [ ]:
# Create folder data test
test_path = ''
os.mkdir(test_path)

In [ ]:
# Move data to test folder
src = ''
des = test_path
move_data_to_path(src, des)

In [ ]:
# Create folder data test
test_path = ''
os.mkdir(test_path)

In [ ]:
# Move data to test folder
src = ''
des = test_path
move_data_to_path(src, des)

# Plot some face crop in dataset

In [ ]:
import matplotlib.pyplot as plt
import cv2

def extract_face(filename,embedder,required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    results = embedder.extract(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)

    return face_array

def get_list_files(dir):
    list_files = []
    for subdir in os.listdir(dir):
        filepath = dir +'/'+subdir
        for filename in os.listdir(filepath):
            list_files.append(filepath+'/'+filename)
    return list_files

In [ ]:
import random

embedder = FaceNet()
plt.subplots(figsize=(20,20))
k = 1
image_list = get_list_files('/content/drive/MyDrive/VietNam/VN_Female')
random_image_list = random.choices(image_list, k = 12)
for image in random_image_list:
    im = extract_face(image,embedder)
    plt.subplot(3,4,k)
    plt.imshow(im)
    plt.axis("off")
    k += 1

# Prepare dataset for tranning

In [ ]:
# Gets a detection dict for each face
# in an image. Each one has the bounding box and
# face landmarks (from mtcnn.MTCNN) along with
# the embedding from FaceNet.
# return List of embedded images in folder
def get_embedding(dir, embedder):
    x = []
    for filename in listdir(dir):
        # set up file path
        filepath = os.path.join(dir,filename) 
        # load image from file
        image = Image.open(filepath)
        # convert to RGB, if needed
        image = image.convert('RGB')
        # convert to array
        pixels = np.asarray(image)
        # get embeddings for the faces in an image
        detections = embedder.extract(pixels, threshold=0.95)
        if(len(detections)<1):
            print('ERROR: Can\'t not detect face: ',filepath)
            continue
        if(len(detections)>1):
            print('ERROR: More than two face detected: ', filepath)
            continue
        x.append(detections[0]['embedding'])
    return x


In [ ]:
# Get embedded list images and list labels in folder
def load_dataset(dir, embedder):
    x , y = [],[]
    for subdir in listdir(dir):
        subdir_path = dir+'/'+subdir
        
        data_embedding = get_embedding(subdir_path, embedder)
        for data in data_embedding:
            x.append(data)
            y.append(subdir)
    return x, y

In [ ]:
# Create embedder tools with keras_facenet
embedder = FaceNet()

In [ ]:
# Create train dataset
dataset_train_path= '/content/drive/MyDrive/Dataset/VN_Female'
train_x, train_y = load_dataset(dataset_train_path, embedder)

In [ ]:
# Create test dataset
dataset_test_path= '/content/drive/MyDrive/Dataset/VN_Female_Test'
test_x, test_y = load_dataset(dataset_test_path, embedder)

# Train model

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(train_y)
train_y = out_encoder.transform(train_y)
test_y = out_encoder.transform(test_y)

In [ ]:
# tranning model
model = SVC(kernel='linear', probability=True)
model.fit(train_x, train_y)
# score
score_train = model.score(train_x, train_y)
score_test = model.score(test_x, test_y)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=100.000, test=100.000


# Use model to recognize faces in images

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
# read image from file path
image = cv2.imread('/content/data_jsc/test/vy/vy10.jpg')
# convert to RGB, if needed
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# convert to array
pixels = asarray(image)
# create the detector, using default weights
detections = embedder.extract(pixels)

# loop through each face in detections
for detect in detections:
    # get embedded image
    embedded_img = detect['embedding']
    # get coordinates (x,y) and weight, height (w, h) of the bounding box
    x,y,w,h= detect['box']

    # expand dimension for feeding data to the model
    samples = np.expand_dims(embedded_img, axis=0)

    # get predict from model
    yhat_class = model.predict(samples)
    yhat_prob = model.predict_proba(samples)

    # get label name
    class_index = yhat_class[0]
    predict_names = out_encoder.inverse_transform(yhat_class)[0]

    # get probability
    class_probability = round(yhat_prob[0,class_index] * 100)
    

    # draw bounding box with label in the image
    start_point = (x,y)
    end_point = (x+w,y+h)  
    # green color in BGR
    color = (0, 255, 0)
    # thickness
    thickness = 3
    # Draw a rectangle with green line borders of thickness of 3 px
    cv2.rectangle(image, start_point, end_point, color, thickness)

    # Put label name if probability >50% and 'Unknown'' if <= 50%
    if class_probability>50:
        cv2.putText(image, "{}:{:.0f}%".format(predict_names,round(class_probability)), (start_point),
          cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
    else:
        cv2.putText(image, "Unknown".format(predict_names,round(class_probability)), (start_point),
          cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

#show image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image, (900,600))
cv2_imshow(image)

## Install tensorflow version 1.* and scipy version 1.1.0 for facenet training

In [ ]:
!pip install -U --pre tensorflow=="1.*"

     |████████████████████████████████| 110.5MB 46kB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 20.1MB 11.5MB/s 
     |████████████████████████████████| 3.8MB 41.0MB/s 
     |████████████████████████████████| 512kB 37.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=c97b6547dce5a070f567ecdf726e25caa0c47676549d2cfd7361be72d02d4490
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninst

In [ ]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 148kB/s 
ERROR: pymc3 3.11.2 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


## Create copy facenet model and set up enviroment

In [ ]:
!git clone https://github.com/davidsandberg/facenet.git

Cloning into 'facenet'...
remote: Enumerating objects: 3149, done.
remote: Total 3149 (delta 0), reused 0 (delta 0), pack-reused 3149
Receiving objects: 100% (3149/3149), 2.94 MiB | 15.19 MiB/s, done.
Resolving deltas: 100% (2230/2230), done.


In [ ]:
%set_env PYTHONPATH=/content/facenet/src:/env/python

env: PYTHONPATH=/content/facenet/src:/env/python


In [ ]:
%%bash
cd drive
cd MyDrive
cd Dataset
mkdir mtcnnpy_182_dataset

mkdir: cannot create directory ‘mtcnnpy_182_dataset’: File exists


## Create data with mtcnn

In [ ]:
# Code for fix bug
# replace line 85 with this for fix code - detect_face.py
# # save np.load
# np_load_old = np.load

# # modify the default parameters of np.load
# np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# # call load_data with allow_pickle implicitly set to true
# data_dict = np.load(data_path, encoding='latin1').item() #pylint: disable=no-member
# # restore np.load for future normal usage
# np.load = np_load_old
!python facenet/src/align/align_dataset_mtcnn.py /content/drive/MyDrive/Dataset/VN_Female /content/drive/MyDrive/Dataset/mtcnnpy_182_dataset --image_size 182 --margin 44

Creating networks and loading parameters



2021-05-16 08:20:35.703838: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-05-16 08:20:35.707998: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-05-16 08:20:35.708316: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5616abbdea00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-16 08:20:35.708356: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version




Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

/content/drive/MyDrive/Dataset/VN_Female/An_Nguy/An_Nguy (146).jpg
facenet/src/align/align_dataset_mtcnn.py:83: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  img

## Training model with tripletloss

In [ ]:
# Code for fix bug
# image = tf.to_float(image) # put code between line 118, and 120 in train_tripletloss.py
!python /content/facenet/src/train_tripletloss.py --logs_base_dir ~/logs/facenet/ --models_base_dir /content/20180402-114759 --data_dir /content/train_dataset/mtcnnpy_182_dataset --image_size 160 --model_def models.inception_resnet_v1 --optimizer RMSPROP --learning_rate 0.01 --weight_decay 1e-4 --max_nrof_epochs 500 --people_per_batch 45 --images_per_person 40

# Install tensorflow version 2.*

In [ ]:
!pip install -U --pre tensorflow=="2.*"

# Test

In [4]:
model = FaceNet()

In [ ]:
model.model.summary()

In [ ]:
def get_data_train(src):
    x , y = [],[]
    list_dir = os.listdir(src)
    for sub_dir in list_dir:
        list_image_path = os.path.join(src,sub_dir)
        list_image = os.listdir(list_image_path)

        with tqdm(total=len(list_image), file=sys.stdout) as pbar:
            for file_path in list_image:
                image_path = os.path.join(list_image_path, file_path)
                image = Image.open(image_path)
                # convert to RGB, if needed
                image = image.convert('RGB')
                # convert to array
                pixels = np.asarray(image)
                x.append(pixels)
                y.append(sub_dir)
                pbar.update(1)
    return np.array(x),y

In [ ]:
src = '/content/drive/MyDrive/Dataset/mtcnnpy_182_dataset'
train_x, train_y = get_data_train(src)

In [31]:
print(train.shape, len(train_y))

(4699, 182, 182, 3) 4699


In [13]:
!pip install -q -U tensorflow-addons

In [16]:
import tensorflow_addons as tfa

model.model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [32]:
gen_train = tf.data.Dataset.from_tensor_slices((train, train_y)).repeat().shuffle(1024).batch(32)

<BatchDataset shapes: ((None, 182, 182, 3), (None,)), types: (tf.uint8, tf.string)>

In [ ]:
history = model.model.fit(gen_train, steps_per_epoch=100, epochs=1000)